In [1]:
from pathlib import Path
import pandas as pd

In [2]:
hdf = pd.read_csv(Path.cwd() / 'data' / 'historical_prices.csv', parse_dates=['_date'], date_format='%Y-%m-%d')
hdf['_year'] = hdf['_date'].dt.year
hdf['_month'] = hdf['_date'].dt.month
hdf = hdf[(hdf['_date'] >= '2012-01-01') & (hdf['_date'] <= '2024-04-30')]

In [3]:
# meta = pd.read_csv(Path.cwd() / 'data' / 'meta.csv', parse_dates=['first_include'], date_format='%Y-%m-%d')

In [3]:
historical = hdf # pd.merge(hdf, meta[['_code', 'first_include']], how='left', on='_code')

In [4]:
# historical = historical[historical['_date'] >= historical['first_include']].drop('first_include', axis=1).reset_index(drop=True)

In [5]:
def monthly_rtn(ts: pd.Series):
    return (ts.iloc[-1] - ts.iloc[0]) / ts.iloc[0]

def monthly_high_end_rtn(ts: pd.Series):
    mx = ts.max()
    return (ts.iloc[-1] - mx) / mx

def monthly_start_high_rtn(ts: pd.Series):
    mx = ts.max()
    return (mx - ts.iloc[0]) / ts.iloc[0]

def daily_rtn_avg(ts: pd.Series):
    return ts.pct_change(1).dropna().mean()

def daily_vola(ts: pd.Series):
    return ts.pct_change(1).dropna().std()

def daily_dvola(ts: pd.Series):
    r = ts.pct_change(1).dropna()
    return r[r < 0].std()

def monthly_mdd(ts: pd.Series):
    r = ts.pct_change(1).dropna()
    comp_ret = (r+1).cumprod()
    peak = comp_ret.expanding(min_periods=1).max()
    dd = (comp_ret/peak)-1
    return abs(dd.min())

In [6]:
monthly_historical = historical.groupby(['_code', '_year', '_month'], as_index=False)['_value'].agg(
    [
        monthly_rtn, 
        monthly_high_end_rtn, 
        monthly_start_high_rtn, 
        daily_rtn_avg, 
        daily_vola, 
        daily_dvola, 
        monthly_mdd
     ]
)

In [7]:
monthly_historical.to_csv(Path.cwd() / 'data' / 'historical_prices_monthly_stat.csv', index=False)